In [1]:
#!/usr/bin/env python
# # -*- coding: utf-8 -*-
# """
# Created on Sun Jun 23 15:14:45 2024
# @author: Mazhar
# """

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pwd

'/content'

In [4]:
try:
    import geopandas as gpd
    import pandas as pd
    import numpy as np
    from pykrige.ok import OrdinaryKriging
    from tqdm import tqdm
    import sqlite3
    import os
    import sys
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        !apt install libspatialindex-dev -qq
        !pip install fiona shapely pyproj rtree --quiet
        !pip install geopandas --quiet
        !pip install pykrige --quiet
        !pip install tqdm --quiet
        !pip install pysqlite3 --quiet
        !pip install pandas --quiet
        !pip install numpy --quiet
        !pip install osmnx --quiet
        !pip install pyproj --quiet
        !pip install rtree --quiet
        !pip install fiona --quiet
        !pip install shapely --quiet
    else:
        print('geopandas not found, please install via conda in your environment')

The following additional packages will be installed:
  libspatialindex-c6 libspatialindex6
The following NEW packages will be installed:
  libspatialindex-c6 libspatialindex-dev libspatialindex6
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 319 kB of archives.
After this operation, 1,416 kB of additional disk space will be used.
Selecting previously unselected package libspatialindex6:amd64.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../libspatialindex6_1.9.3-2_amd64.deb ...
Unpacking libspatialindex6:amd64 (1.9.3-2) ...
Selecting previously unselected package libspatialindex-c6:amd64.
Preparing to unpack .../libspatialindex-c6_1.9.3-2_amd64.deb ...
Unpacking libspatialindex-c6:amd64 (1.9.3-2) ...
Selecting previously unselected package libspatialindex-dev:amd64.
Preparing to unpack .../libspatialindex-dev_1.9.3-2_amd64.deb ...
Unpacking libspatialindex-dev:amd64 (1.9.3-2) ...
Setting up libspatialindex6:a

In [5]:
import geopandas as gpd
import pandas as pd
import numpy as np
from pykrige.ok import OrdinaryKriging
from tqdm import tqdm
import osmnx as ox
import pyproj
import sqlite3
import os
import sys

#### VARS

In [6]:
ZONE = 9

#####################
# BUFFER_RADIUS_01 = 10
# BUFFER_RADIUS_02 = 15
# BUFFER_RADIUS_03 = 15
#####################

SAMPLE_FRACTION = 0.0033
# SAMPLE_FRACTION = 0.0028
# SAMPLE_FRACTION = 0.04
LAYER_NAME = "spatial_Joins"

SJOIN_CHUNK_SIZE = 2000

SAVE_FILE_CSV_V1 = f"/content/drive/MyDrive/IPAUA_Maz/dataset/zone_{ZONE}_v1.csv"
SAVE_FILE_CSV_V2 = f"/content/drive/MyDrive/IPAUA_Maz/dataset/zone_{ZONE}_v2.csv"
SAVE_FILE_GPKG_V1 = f"/content/drive/MyDrive/IPAUA_Maz/dataset/zone_{ZONE}_v1.gpkg"
SAVE_FILE_GPKG_V2 = f"/content/drive/MyDrive/IPAUA_Maz/dataset/zone_{ZONE}_v2.gpkg"

In [7]:
PREFIX = f"/content/drive/MyDrive/IPAUA_Maz/csv_input/zone{ZONE}"

FILE_1 = f"{PREFIX}/soil_moisture_zone{ZONE}_median.csv"
FILE_2 = f"{PREFIX}/NDBI_NDVI_BUILT_zone{ZONE}.csv"
FILE_3 = f"{PREFIX}/land_surface_roughness_zone{ZONE}.csv"
FILE_4 = f"{PREFIX}/land_slope_zone{ZONE}.csv"
FILE_5 = f"{PREFIX}/zone{ZONE}_LST_UHI_UTFVI_2023-01-01_2024-01-01.csv"
FILE_6 = f"{PREFIX}/ndwi_10m_zone{ZONE}.csv"
FILE_7 = f"{PREFIX}/zone{ZONE}_savi_median_2023-01-01_2024-01-01.csv"
FILE_8 = f"{PREFIX}/lulc_zone{ZONE}.csv"
FILE_9 = f"{PREFIX}/amenity_zone{ZONE}.csv"
FILE_10 = f"{PREFIX}/land_use_zone{ZONE}.csv"
FILE_11 = f"{PREFIX}/solar_exposure_zone{ZONE}.csv"
FILE_12 = f"{PREFIX}/zone{ZONE}_median_air_quality_merged.csv"

# File paths
files = [FILE_1, FILE_2, FILE_3, FILE_4, FILE_5, FILE_6, FILE_7, FILE_8, FILE_9, FILE_10, FILE_11, FILE_12]

# Columns to drop for specific files
columns_to_drop = {
    FILE_2: ["NDVI"],
    FILE_5: ["FV", "EM", "THERMAL"],
    # FILE_10: ["Unnamed: 0"],
    # FILE_11: ["Unnamed: 0"],
    # FILE_12: ["Unnamed: 0"],
}

#### FUNCTIONS

In [8]:
# HANDLE MISSING VALUES
def kriging_interpolation_subset(df, column, x_col, y_col, sample_fraction=0.1):
    # Subset the data to a manageable size
    sample_df = df.dropna(subset=[column]).sample(frac=sample_fraction, random_state=1)

    known_values = sample_df[column].values
    known_coords = sample_df[[x_col, y_col]].values
    missing_coords = df[df[column].isna()][[x_col, y_col]].values

    # Perform Ordinary Kriging
    kriging = OrdinaryKriging(known_coords[:, 0], known_coords[:, 1], known_values,
                              variogram_model='linear', verbose=False, enable_plotting=False)
    interpolated_values, ss = kriging.execute('points', missing_coords[:, 0], missing_coords[:, 1])

    # Fill the missing values in the DataFrame
    df.loc[df[column].isna(), column] = interpolated_values
    return df

# Usage
# df6 = kriging_interpolation_subset(df6, 'NDWI', 'Longitude', 'Latitude', sample_fraction=0.1)

In [9]:
# Function to perform spatial join using gpd.sjoin_nearest in chunks
def spatial_join_chunks(merged_gdf, gdf, chunk_size=1000):
    # Create spatial index for the gdf
    sindex = gdf.sindex

    chunks = []
    for i in range(0, len(merged_gdf), chunk_size):
        chunk = merged_gdf.iloc[i:i + chunk_size]
        # Calculate the bounding box of the chunk
        bbox = chunk.geometry.total_bounds
        # Find possible matches within the bounding box
        possible_matches_index = list(sindex.intersection(bbox))
        possible_matches = gdf.iloc[possible_matches_index]

        # Perform the nearest spatial join
        joined_chunk = gpd.sjoin_nearest(chunk, possible_matches, how="left")
        chunks.append(joined_chunk)

    # Concatenate all the chunks back together
    merged_gdf = pd.concat(chunks, ignore_index=True)
    merged_gdf = merged_gdf.drop(columns=["index_right"], errors='ignore')

    return merged_gdf

In [10]:
# Reading .gpkg file in chunks
def read_gpkg_in_chunks(gpkg_path, layer_name, chunk_size=10000):
    # Connect to the GeoPackage using sqlite3
    conn = sqlite3.connect(gpkg_path)

    # Get the total number of rows
    query_count = f"SELECT COUNT(*) FROM {layer_name}"
    total_rows = pd.read_sql(query_count, conn).iloc[0, 0]

    print(f"Total rows to read: {total_rows}")

    chunks = []
    offset = 0

    while offset < total_rows:
        query = f"SELECT * FROM {layer_name} LIMIT {chunk_size} OFFSET {offset}"
        chunk = pd.read_sql(query, conn)

        # Convert the DataFrame chunk to a GeoDataFrame
        gdf_chunk = gpd.GeoDataFrame(
            chunk, geometry=gpd.points_from_xy(chunk['Longitude'], chunk['Latitude']), crs="EPSG:4326"
        )
        chunks.append(gdf_chunk)

        offset += chunk_size
        print(f"\rRead {offset} rows", end='', flush=True)

    conn.close()

    # Concatenate all chunks into a single GeoDataFrame
    full_gdf = pd.concat(chunks, ignore_index=True)

    return full_gdf

#### LOAD DATA

In [11]:
# Load the CSV files into DataFrames
dataframes = []
for file in tqdm(files, colour='green', desc="Generating DataFrames"):
    df = pd.read_csv(file)
    if file in columns_to_drop:
        df = df.drop(columns=columns_to_drop[file])
    if file == FILE_6:
        df = kriging_interpolation_subset(df, 'NDWI', 'Longitude', 'Latitude', sample_fraction=SAMPLE_FRACTION)
    dataframes.append(df)

Generating DataFrames: 100%|██████████| 12/12 [00:52<00:00,  4.39s/it]


#### CREATE GEO DATAFRAMES

In [12]:
# Function to clean and convert coordinates
def preprocess_coordinates(df, x_col='Longitude', y_col='Latitude'):
    df[x_col] = df[x_col].apply(lambda x: x[0] if isinstance(x, list) else x)
    df[y_col] = df[y_col].apply(lambda y: y[0] if isinstance(y, list) else y)
    df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
    df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
    return df.dropna(subset=[x_col, y_col])

In [13]:
# Clean and convert coordinates in each DataFrame
cleaned_dataframes = [preprocess_coordinates(df) for df in dataframes]

In [14]:
gdfs = [gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["Longitude"], df["Latitude"]), crs="EPSG:4326") for df in dataframes]

In [15]:
# Print the first few rows of the first GeoDataFrame
print(gdfs[9].head())

    Latitude  Longitude   Zone    Land-Use                  geometry
0  45.535156   9.164591  zone9  industrial  POINT (9.16459 45.53516)
1  45.520352   9.217235  zone9  industrial  POINT (9.21724 45.52035)
2  45.520075   9.218253  zone9  industrial  POINT (9.21825 45.52008)
3  45.520018   9.218401  zone9  industrial  POINT (9.21840 45.52002)
4  45.519983   9.218574  zone9  industrial  POINT (9.21857 45.51998)


In [16]:
# Drop "Latitude", "Longitude", and "Zone" columns from each GeoDataFrame
for i, gdf in enumerate(tqdm(gdfs, colour='blue', desc="Droping Columns")):
    gdfs[i] = gdf.drop(columns=["Latitude", "Longitude", "Zone"])

Droping Columns: 100%|██████████| 12/12 [00:00<00:00, 71.73it/s]


In [17]:
# Reproject all GeoDataFrames to a common CRS (UTM zone 32N for the Milan area)
gdfs = [gdf.to_crs(epsg=32632) for gdf in gdfs]

#### PERFORM SPATIAL JOINS | USING gpd.sjoin_nearest()

---



In [18]:
# Perform spatial join iteratively with chunking and spatial index
merged_gdf = gdfs[0]
for gdf in tqdm(gdfs[1:], colour='green', desc="Performing Spatial Joins"):
    merged_gdf = spatial_join_chunks(merged_gdf, gdf, chunk_size=SJOIN_CHUNK_SIZE)

print("Spatial joins completed.")

Performing Spatial Joins: 100%|██████████| 11/11 [03:58<00:00, 21.67s/it]

Spatial joins completed.


#### **PERFORM SPATIAL JOINS | USING gpd.sjoin()**

---



##### Setting BUFFERS for gpd.sjoin()

In [19]:
# # # Buffer the geometries of all other GeoDataFrames except the first one
# # for gdf in tqdm(gdfs[1:], colour='green', desc="Setting Up Buffer Radius 01"):
# #     gdf['geometry'] = gdf.geometry.buffer(BUFFER_RADIUS_01)  # Buffer size in meters

# # Buffer the geometries of all other GeoDataFrames except the first one
# for gdf in tqdm(gdfs[1:5], colour='green', desc="Setting Up Buffer Radius 01"):
#     gdf['geometry'] = gdf.geometry.buffer(BUFFER_RADIUS_01)  # Buffer size in meters

# # Buffer the geometries of all other GeoDataFrames except the first one
# for gdf in tqdm(gdfs[5:6], colour='green', desc="Setting Up Buffer Radius 01"):
#     gdf['geometry'] = gdf.geometry.buffer(BUFFER_RADIUS_02)  # Buffer size in meters

# # Buffer the geometries of all other GeoDataFrames except the first one
# for gdf in tqdm(gdfs[6:7], colour='green', desc="Setting Up Buffer Radius 01"):
#     gdf['geometry'] = gdf.geometry.buffer(BUFFER_RADIUS_01)  # Buffer size in meters

# # Buffer the geometries of all other GeoDataFrames except the first one
# for gdf in tqdm(gdfs[7:8], colour='green', desc="Setting Up Buffer Radius 01"):
#     gdf['geometry'] = gdf.geometry.buffer(BUFFER_RADIUS_02)  # Buffer size in meters

# for gdf in tqdm(gdfs[8:], colour='blue', desc="Setting Up Buffer Radius 02"):
#     gdf['geometry'] = gdf.geometry.buffer(BUFFER_RADIUS_03)  # Buffer size in meters

##### Perform Spatial Join using gpd.sjoin()

In [20]:
# # Perform spatial join iteratively with chunking and spatial index
# merged_gdf = gdfs[0]
# for gdf in tqdm(gdfs[1:], colour='green', desc="Performing Spatial Joins"):
#     # Create spatial index
#     sindex = gdf.sindex

#     # Perform spatial join in chunks
#     chunk_size = 1000
#     chunks = []
#     for i in range(0, len(merged_gdf), chunk_size):
#         chunk = merged_gdf.iloc[i:i+chunk_size]
#         possible_matches_index = list(sindex.intersection(chunk.geometry.total_bounds))
#         possible_matches = gdf.iloc[possible_matches_index]
#         joined_chunk = gpd.sjoin(chunk, possible_matches, how="left", predicate="intersects")
#         chunks.append(joined_chunk)

#     merged_gdf = pd.concat(chunks, ignore_index=True)
#     merged_gdf = merged_gdf.drop(columns=["index_right"], errors='ignore')

#### CLEAN UP

In [21]:
print("Wait..........Finishing UP")
# Reproject the merged GeoDataFrame back to geographic CRS
merged_gdf = merged_gdf.to_crs(epsg=4326)

# Drop the geometry column if no longer needed
# merged_gdf = merged_gdf.drop(columns='geometry')

# Drop duplicates only if all values in the row are duplicates
merged_gdf = merged_gdf.drop_duplicates()

# Drop rows where 'Value' in column is null
merged_gdf = merged_gdf.dropna(how="all", subset=merged_gdf.columns[:-2].to_list())

# Extract the Longitude and Latitude from the geometry column
merged_gdf['Longitude'] = merged_gdf.geometry.x
merged_gdf['Latitude'] = merged_gdf.geometry.y

# Move the geometry column to the last position
columns = [col for col in merged_gdf.columns if col != 'geometry'] + ['geometry']
merged_gdf = merged_gdf[columns]

Wait..........Finishing UP


In [22]:
merged_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1012174 entries, 0 to 1134632
Data columns (total 24 columns):
 #   Column                                   Non-Null Count    Dtype   
---  ------                                   --------------    -----   
 0   soil_moisture                            1012174 non-null  float64 
 1   NDBI                                     1012174 non-null  float64 
 2   BU                                       1012174 non-null  float64 
 3   Roughness                                1012174 non-null  int64   
 4   Slope                                    1012174 non-null  float64 
 5   NDVI                                     1012174 non-null  float64 
 6   LST                                      1012174 non-null  float64 
 7   UHI                                      1012174 non-null  float64 
 8   UTFVI                                    1012174 non-null  float64 
 9   NDWI                                     1012174 non-null  float64 
 10  SAV

In [23]:
# Summarize NaN values for each column
nan_summary = merged_gdf.isna().sum()
nan_summary

soil_moisture                                   0
NDBI                                            0
BU                                              0
Roughness                                       0
Slope                                           0
NDVI                                            0
LST                                             0
UHI                                             0
UTFVI                                           0
NDWI                                            0
SAVI                                            0
lulc_classes                                    0
Amenity                                       384
Land-Use                                        0
GHI (kWh/m2)                                 6214
CH4_column_volume_mixing_ratio_dry_air     875263
CO_column_number_density                   875263
tropospheric_HCHO_column_number_density    875263
NO2_column_number_density                  875263
O3_column_number_density                   875263


In [24]:
merged_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [25]:
# Show the merged DataFrame
# print(merged_gdf)

#### SAVING TO .CSV

In [26]:
# Save the merged GeoDataFrame to a CSV file
print(f"DATA IS BEING SAVED TO .CSV FILE ........Please Wait")
merged_gdf.to_csv(SAVE_FILE_CSV_V2, index=False)
print(f"DATA SAVE TO {SAVE_FILE_CSV_V2}")

# Get the file size in bytes
file_size_bytes = os.path.getsize(SAVE_FILE_CSV_V2)
# Convert the file size to megabytes
file_size_mb = file_size_bytes / (1024 * 1024)
print(f"File Size of {SAVE_FILE_CSV_V2}: {file_size_mb:.2f} MB")

DATA IS BEING SAVED TO .CSV FILE ........Please Wait
DATA SAVE TO /content/drive/MyDrive/IPAUA_Maz/dataset/zone_9_v2.csv
File Size of /content/drive/MyDrive/IPAUA_Maz/dataset/zone_9_v2.csv: 257.12 MB


#### SAVING TO .GPKG

In [27]:
# Ensure the GeoDataFrame has latitude and longitude columns
# if 'Latitude' not in merged_gdf.columns or 'Longitude' not in merged_gdf.columns:
#     raise ValueError("GeoDataFrame must have 'Latitude' and 'Longitude' columns")

# Create a geometry column from latitude and longitude
# merged_gdf['geometry'] = merged_gdf.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
# merged_gdf = merged_gdf.drop(columns=['Longitude', 'Latitude'])

# # Set the CRS
# merged_gdf = gpd.GeoDataFrame(merged_gdf, geometry='geometry', crs="EPSG:4326")

# # Ensure all geometries are valid
# merged_gdf['geometry'] = merged_gdf['geometry'].buffer(0)

# Save to GPKG file
print(f"DATA IS BEING SAVED TO .GPKG FILE ........Please Wait")
merged_gdf.to_file(driver='GPKG', filename=SAVE_FILE_GPKG_V2, layer=LAYER_NAME)
print(f"DATA SAVED TO {SAVE_FILE_GPKG_V2}")

# Get the file size in bytes
file_size_bytes = os.path.getsize(SAVE_FILE_GPKG_V2)
# Convert the file size to megabytes
file_size_mb = file_size_bytes / (1024 * 1024)
print(f"File Size of {SAVE_FILE_GPKG_V2}: {file_size_mb:.2f} MB")

DATA IS BEING SAVED TO .GPKG FILE ........Please Wait
DATA SAVED TO /content/drive/MyDrive/IPAUA_Maz/dataset/zone_9_v2.gpkg
File Size of /content/drive/MyDrive/IPAUA_Maz/dataset/zone_9_v2.gpkg: 240.63 MB


In [28]:
merged_gdf.head()

,soil_moisture,NDBI,BU,Roughness,Slope,NDVI,LST,UHI,UTFVI,NDWI,...,GHI (kWh/m2),CH4_column_volume_mixing_ratio_dry_air,CO_column_number_density,tropospheric_HCHO_column_number_density,NO2_column_number_density,O3_column_number_density,SO2_column_number_density,Longitude,Latitude,geometry
0,45.511588,-0.034360,-0.088197,52,89.990234,0.205030,32.807213,-0.024135,-0.001928,-0.197248,...,1407.3,NaN,NaN,NaN,NaN,NaN,NaN,9.192370,45.480355,POINT (9.19237 45.48036)
1,41.233572,-0.034360,-0.088197,52,89.990234,0.205030,32.807213,-0.024135,-0.001928,-0.273191,...,1407.3,NaN,NaN,NaN,NaN,NaN,NaN,9.192550,45.480355,POINT (9.19255 45.48036)
2,60.945201,-0.052701,-0.134789,1,89.986189,0.289858,32.765621,-0.040007,-0.003200,-0.293753,...,1407.4,NaN,NaN,NaN,NaN,NaN,NaN,9.192730,45.480355,POINT (9.19273 45.48036)
3,60.024595,-0.052701,-0.134789,1,89.984559,0.123400,35.511621,1.007879,0.074375,-0.157025,...,1407.4,NaN,NaN,NaN,NaN,NaN,NaN,9.192909,45.480355,POINT (9.19291 45.48036)
4,42.472361,-0.009582,-0.067800,1,89.984559,0.123400,35.511621,1.007879,0.074375,-0.255411,...,1407.4,NaN,NaN,NaN,NaN,NaN,NaN,9.193089,45.480355,POINT (9.19309 45.48036)
